In [1]:
import pandas as pd

In [2]:
def normalize(x):
    x = x.strip()
    # remove only the FIRST { and LAST }
    if x.startswith("{") and x.endswith("}"):
        x = x[1:-1]
    x = x.strip().strip('"').strip("'")
    return x


def accuracy(row):
    pred = row["pred"]
    true = row["test"]
    if pred == true:
        return 1
    else:
        return 0

In [3]:
pred_df = pd.read_csv("combine_data/mapped_responses.csv")
pred_df.head()

,fdc_id,description,response
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",biscuits
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",cinnamon
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",breadcrumb
3,167515,"George Weston Bakeries, Thomas English Muffins",muffin
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",waffle


In [4]:
test_df = pd.read_csv("data/test/TEST_food_branded.csv", usecols=["fdc_id", "description", "mapped_ingredient"])
test_df.head()

,fdc_id,description,mapped_ingredient
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",buttermilk biscuit
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",cinnamon roll
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",bread crumb
3,167515,"George Weston Bakeries, Thomas English Muffins",english muffin
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",buttermilk waffle


In [5]:
# Merge using test_df as the left df to keep its order
merged_df = test_df[["fdc_id", "mapped_ingredient"]].merge(
    pred_df, on="fdc_id", how="inner"
)
merged_df.head()

,fdc_id,mapped_ingredient,description,response
0,167512,buttermilk biscuit,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",biscuits
1,167513,cinnamon roll,"Pillsbury, Cinnamon Rolls with Icing, refriger...",cinnamon
2,167514,bread crumb,"Kraft Foods, Shake N Bake Original Recipe, Coa...",breadcrumb
3,167515,english muffin,"George Weston Bakeries, Thomas English Muffins",muffin
4,167516,buttermilk waffle,"Waffles, buttermilk, frozen, ready-to-heat",waffle


In [6]:
pred_sets = [normalize(x) for x in merged_df["response"]]
test_sets = [normalize(x) for x in merged_df["mapped_ingredient"]]

new_df = pd.DataFrame({
    "fdc_id": merged_df["fdc_id"],
    "description": merged_df["description"],
    "pred": pred_sets,
    "test": test_sets,
})
new_df.head(300)

,fdc_id,description,pred,test
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",biscuits,buttermilk biscuit
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",cinnamon,cinnamon roll
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",breadcrumb,bread crumb
3,167515,"George Weston Bakeries, Thomas English Muffins",muffin,english muffin
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",waffle,buttermilk waffle
...,...,...,...,...
295,167807,"Cherry juice, tart",juice,cherry juice
296,167808,"Raspberries, puree, seedless",raspberry,raspberry puree
297,167809,"Raspberries, puree, with seeds",raspberry puree,raspberry puree
298,167810,"Pork, fresh, composite of trimmed leg, loin, s...",pork,pork


In [7]:
new_df["accuracy"] = new_df.apply(accuracy, axis=1)

In [8]:
print(f"Accuracy:  {new_df['accuracy'].mean():.4f} (Total Correct Rows / Total Rows)")

Accuracy:  0.3800 (Total Correct Rows / Total Rows)
